In [1]:
import urllib.request
import pandas as pd
from bs4 import BeautifulSoup
from bs4 import UnicodeDammit
import urllib.parse
import requests
import re
import numpy as np
import pandas as pd
from tqdm import tqdm
from itertools import product
np.set_printoptions(threshold = np.inf)
pd.set_option('display.max_row',100000)

# 提取KEGG BRITE文件中的Targets, Drugs, DTIs

In [2]:
with open('./0_database_file/Nuclear Receptors - Homo sapiens (human) + Drug.keg') as f:
    file = f.readlines()
file

['+D\tReceptor\tKO\n',
 '#<h2><a href="/kegg/kegg2.html"><img src="/Fig/bget/kegg3.gif" align="middle" border=0></a> &nbsp; Nuclear Receptors - Homo sapiens (human) + Drug</h2>\n',
 '%<style type="text/css"><!-- #grid{ table-layout:fixed; font-family: monospace; position: relative;width: 1200px; color: black; }.col1{ position: relative; background : white; z-index: 1; width: 600px; overflow: hidden; }.col2{ position: relative; background : white; z-index: 2; padding-left: 10px; }--></style>\n',
 '!\n',
 'A<b>1. Thyroid hormone like</b>\n',
 'B  1A. Thyroid hormone receptor (THR)\n',
 'C    7067 THRA; thyroid hormone receptor alpha\tK05547 THRA; thyroid hormone receptor alpha \n',
 'D      D01010 Levothyroxine sodium (USP)\n',
 'D      D01011 Liothyronine sodium (JP17/USAN)\n',
 'D      D01012 Dried thyroid (JP17)\n',
 'D      D03750 Dextrothyroxine sodium (USAN)\n',
 'D      D06482 Thyroid (USP)\n',
 'D      D07214 Tiratricol (INN)\n',
 'D      D08125 Levothyroxine (BAN)\n',
 'D      D

In [3]:
NR_dict = {}
NR_all_drugs = []
num_DTIs = 0
for line in tqdm(file):
    if line[0] == 'C':
        NR_drug_list = []
        hsa_id = re.findall('C    [0-9]*', line)[0][5:]
        NR_dict[hsa_id] = ''
        
    elif line[0] == 'D':
        drug_id = re.findall('D      D[0-9]*',line)[0][7:]
        NR_drug_list.append(drug_id)
        NR_all_drugs.append(drug_id)
        NR_dict[hsa_id] = NR_drug_list
        num_DTIs += 1
            
NR_dict

100%|██████████| 693/693 [00:00<00:00, 439071.40it/s]


{'7067': ['D01010',
  'D01011',
  'D01012',
  'D03750',
  'D06482',
  'D07214',
  'D08125',
  'D08128',
  'D08129',
  'D11113'],
 '7068': ['D01010',
  'D01011',
  'D01012',
  'D03750',
  'D06482',
  'D07214',
  'D08125',
  'D08128',
  'D08129',
  'D09381',
  'D11113',
  'D11602'],
 '5914': ['D00094',
  'D00164',
  'D00316',
  'D00348',
  'D01132',
  'D01418',
  'D01516',
  'D01621',
  'D02106',
  'D02754',
  'D02815',
  'D04162',
  'D04636',
  'D06543',
  'D08477'],
 '5915': ['D00094',
  'D00164',
  'D00316',
  'D00348',
  'D01112',
  'D01132',
  'D01516',
  'D01621',
  'D02754',
  'D02815',
  'D04162',
  'D04636',
  'D06543',
  'D08477'],
 '5916': ['D00094',
  'D00164',
  'D00316',
  'D00348',
  'D01112',
  'D01132',
  'D01516',
  'D01621',
  'D02754',
  'D02815',
  'D04162',
  'D04636',
  'D06543',
  'D08477',
  'D09365',
  'D11225'],
 '5465': ['D00279',
  'D00334',
  'D00565',
  'D01208',
  'D01274',
  'D01366',
  'D03521',
  'D05091',
  'D05397',
  'D06102',
  'D06647',
  'D07187',

In [4]:
DTIs_list = []
for key,value in NR_dict.items():
    if value != '':
        for drug in NR_dict[key]:
            DTIs_list.append((key,drug))
        
assert len(DTIs_list) == num_DTIs

DTIs_np = np.array(DTIs_list)
del DTIs_list

DTIs_np.shape, DTIs_np[:5]

((603, 2), array([['7067', 'D01010'],
        ['7067', 'D01011'],
        ['7067', 'D01012'],
        ['7067', 'D03750'],
        ['7067', 'D06482']], dtype='<U6'))

In [5]:
NR_all_drugs = list(set(NR_all_drugs))
NR_all_protein = list(NR_dict.keys())

print('# all_drugs = ', len(NR_all_drugs))
print('# all_proteins = ', len(NR_all_protein))
print('# all DTIs = ', num_DTIs)

# all_drugs =  461
# all_proteins =  48
# all DTIs =  603


# 合并Yamanishi_08

In [6]:
with open('/home/yanyichu/11_MFDF_code/1_original_data/NR/nr_admat_dgc_mat_2_line.txt', 'r') as f:
    Yam08_nr_DTIs = f.readlines()

Y_DTIs = np.array([item[:-1].split('\thsa') for item in Yam08_nr_DTIs])
Y_drugs = set(Y_DTIs[:, 0])
Y_targets = set(Y_DTIs[:, 1])

print('Yamanishi_08 dataset: #Drugs = {}, #Targets = {}, #DTIs = {}'.format(len(Y_drugs),
                                                                            len(Y_targets), 
                                                                            len(Y_DTIs)))
assert len(Yam08_nr_DTIs) == len(Y_DTIs)

Yamanishi_08 dataset: #Drugs = 54, #Targets = 26, #DTIs = 90


In [7]:
NR_exsit_drugs = list(set(DTIs_np[:,1]))
NR_exsit_targets = list(set(DTIs_np[:,0]))
print('KEGG BRITE FILE(exist DTIs): #Drugs = {}, #Targets = {}, #DTIs = {}'.format(len(NR_exsit_drugs),
                                                                                   len(NR_exsit_targets), 
                                                                                   len(DTIs_np)))

KEGG BRITE FILE(exist DTIs): #Drugs = 461, #Targets = 19, #DTIs = 603


In [8]:
Union_Drugs = set(Y_drugs).union(set(NR_all_drugs))
Union_Targets = set(Y_targets).union(set(NR_all_protein))
print('ALL: After combination of KEGG BRITE & Yamanishi: #Drugs = {}, #Targets = {}'.format(len(Union_Drugs), len(Union_Targets)))

ALL: After combination of KEGG BRITE & Yamanishi: #Drugs = 470, #Targets = 48


In [9]:
Y_DTIs_df = pd.DataFrame(Y_DTIs, columns = ['KEGG Drug ID','KEGG_hsa_ID'])
Y_DTIs_df.head()

,KEGG Drug ID,KEGG_hsa_ID
0,D00094,190
1,D00066,2099
2,D00067,2099
3,D00105,2099
4,D00312,2099


In [10]:
KEGG_DTIs_df = pd.DataFrame(DTIs_np, columns = ['KEGG_hsa_ID', 'KEGG Drug ID'])
KEGG_DTIs_df.head()

,KEGG_hsa_ID,KEGG Drug ID
0,7067,D01010
1,7067,D01011
2,7067,D01012
3,7067,D03750
4,7067,D06482


In [11]:
KEGG_Yamanishi_merged_DTIs = pd.concat([KEGG_DTIs_df, Y_DTIs_df], sort=False).drop_duplicates()

print('KEGG_Yamanishi_merged_NR_DTIs: #DTIs = {}, #Drugs = {}, #Targets = {}'.format(KEGG_Yamanishi_merged_DTIs.shape[0],
                                                                             len(set(KEGG_Yamanishi_merged_DTIs.iloc[:,1])),
                                                                             len(set(KEGG_Yamanishi_merged_DTIs.iloc[:,0]))))
KEGG_Yamanishi_merged_DTIs.head()

KEGG_Yamanishi_merged_NR_DTIs: #DTIs = 632, #Drugs = 470, #Targets = 29


,KEGG_hsa_ID,KEGG Drug ID
0,7067,D01010
1,7067,D01011
2,7067,D01012
3,7067,D03750
4,7067,D06482


# 对48个Targets进行KEGG爬虫，查找其Uniprot ID

In [12]:
def scrapy_uniprot_id_from_kegg(hsa_id = '10894'):
    
    url = 'https://www.kegg.jp/dbget-bin/www_bget?hsa:' + str(hsa_id)
    html = requests.get(url).content.decode()
    soup = BeautifulSoup(html, 'lxml')
    
    href_link = soup.select('td > table > tr > td > a')

    uniprot_id_list = []
    for item in href_link:

        try:
            item_link = item.get('href')
            if item_link[:31] == 'https://www.uniprot.org/uniprot':
                uniprot_id_list.append(item.text)

        except:
            continue
    
    return uniprot_id_list

In [13]:
uniprot_target_id_dict = {}
all_uniprot_id = []

for target in tqdm(list(Union_Targets)):
    
    try:
        item = scrapy_uniprot_id_from_kegg(hsa_id = target)
        uniprot_target_id_dict[target] = item
        all_uniprot_id.extend(item)
        
    except:
        print('ERROR: ', target)

  4%|▍         | 2/48 [00:30<12:48, 16.71s/it]

ERROR:  5467


100%|██████████| 48/48 [14:45<00:00, 18.45s/it]


In [15]:
for target in tqdm(list(['5467'])):
    
    try:
        item = scrapy_uniprot_id_from_kegg(hsa_id = target)
        uniprot_target_id_dict[target] = item
        all_uniprot_id.extend(item)
        
    except:
        print('ERROR: ', target)

100%|██████████| 1/1 [00:09<00:00,  9.60s/it]


In [16]:
uniprot_target_id_dict

{'2100': ['Q92731', 'Q7LCB3'],
 '4929': ['P43354', 'F1D8N6', 'Q53EL4'],
 '5468': ['P37231'],
 '9971': ['Q96RI1'],
 '6097': ['P51449', 'Q6I9R9'],
 '2908': ['P04150', 'F1D8N4'],
 '4306': ['P08235', 'B0ZBF6'],
 '8013': ['Q92570', 'A0A024R168'],
 '2649': ['Q15406', 'F1D8S0'],
 '3172': ['P41235', 'F1D8T1'],
 '7101': ['Q9Y466', 'B6ZGT9'],
 '6256': ['P19793', 'F1D8Q5', 'Q6P3U7'],
 '10002': ['Q9Y5X4'],
 '3164': ['P22736', 'A0A024R126'],
 '7376': ['P55055', 'F1D8P7'],
 '6257': ['P28702', 'Q5STP9'],
 '2101': ['P11474', 'Q569H8'],
 '9572': ['P20393', 'F1D8S3'],
 '8856': ['O75469'],
 '8431': ['Q15466'],
 '2516': ['Q13285', 'F1D8R8'],
 '7067': ['P10827'],
 '7026': ['P24468', 'F1D8R0'],
 '6095': ['P35398'],
 '2104': ['P62508', 'F1D8R5'],
 '10062': ['Q13133', 'B4DXU5', 'F1D8N1'],
 '2063': ['P10588', 'F1D8R3'],
 '7182': ['P49116'],
 '7181': ['P13056', 'H9NIM2'],
 '5915': ['P10826'],
 '5916': ['P13631', 'A8K3H3'],
 '2099': ['P03372', 'G4XH65'],
 '190': ['P51843', 'F1D8P4'],
 '6258': ['P48443', 'F1D8Q7'

In [14]:
uniprot_target_id_dict = {'2100': ['Q92731', 'Q7LCB3'],
 '4929': ['P43354', 'F1D8N6', 'Q53EL4'],
 '5468': ['P37231'],
 '9971': ['Q96RI1'],
 '6097': ['P51449', 'Q6I9R9'],
 '2908': ['P04150', 'F1D8N4'],
 '4306': ['P08235', 'B0ZBF6'],
 '8013': ['Q92570', 'A0A024R168'],
 '2649': ['Q15406', 'F1D8S0'],
 '3172': ['P41235', 'F1D8T1'],
 '7101': ['Q9Y466', 'B6ZGT9'],
 '6256': ['P19793', 'F1D8Q5', 'Q6P3U7'],
 '10002': ['Q9Y5X4'],
 '3164': ['P22736', 'A0A024R126'],
 '7376': ['P55055', 'F1D8P7'],
 '6257': ['P28702', 'Q5STP9'],
 '2101': ['P11474', 'Q569H8'],
 '9572': ['P20393', 'F1D8S3'],
 '8856': ['O75469'],
 '8431': ['Q15466'],
 '2516': ['Q13285', 'F1D8R8'],
 '7067': ['P10827'],
 '7026': ['P24468', 'F1D8R0'],
 '6095': ['P35398'],
 '2104': ['P62508', 'F1D8R5'],
 '10062': ['Q13133', 'B4DXU5', 'F1D8N1'],
 '2063': ['P10588', 'F1D8R3'],
 '7182': ['P49116'],
 '7181': ['P13056', 'H9NIM2'],
 '5915': ['P10826'],
 '5916': ['P13631', 'A8K3H3'],
 '2099': ['P03372', 'G4XH65'],
 '190': ['P51843', 'F1D8P4'],
 '6258': ['P48443', 'F1D8Q7'],
 '2103': ['O95718'],
 '7025': ['P10589'],
 '3174': ['Q14541'],
 '5465': ['Q07869', 'F1D8S4'],
 '5467': ['Q03181', 'A0A024RCW6'],
 '9970': ['Q14994'],
 '7068': ['P10828', 'F1D8N7'],
 '5914': ['P10276', 'Q6I9R7'],
 '367': ['P10275', 'Q9NUA2'],
 '2494': ['O00482'],
 '6096': [],
 '7421': ['P11473', 'F1D8P8'],
 '5241': ['P06401'],
 '9975': ['Q14995', 'F1D8P2']}

In [17]:
hsa_uniprot_id_list = []
for key, value in tqdm(uniprot_target_id_dict.items()):
    for v in value:
        hsa_uniprot_id_list.append([key, v])
        
hsa_uniprot_id_df = pd.DataFrame(hsa_uniprot_id_list, columns = ['KEGG_hsa_ID', 'UniProt ID'])
print(hsa_uniprot_id_df.shape)
hsa_uniprot_id_df.head()

100%|██████████| 48/48 [00:00<00:00, 80854.05it/s]

(82, 2)


,KEGG_hsa_ID,UniProt ID
0,2100,Q92731
1,2100,Q7LCB3
2,4929,P43354
3,4929,F1D8N6
4,4929,Q53EL4


# 对48个Targets进行DrugBank比对

### 读取DrugBank文件

######  DB_drug_links

In [18]:
DB_drug_links = pd.read_csv('./0_database_file/drug links.csv')
print(DB_drug_links.shape)
print(DB_drug_links.columns)

DB_drug_links = DB_drug_links[DB_drug_links['Drug Type'] == 'SmallMoleculeDrug'][['DrugBank ID', 'KEGG Drug ID']].reset_index(drop = True)
print(DB_drug_links.shape)
DB_drug_links.head()

(3883, 23)
Index(['DrugBank ID', 'Name', 'CAS Number', 'Drug Type', 'KEGG Compound ID',
       'KEGG Drug ID', 'PubChem Compound ID', 'PubChem Substance ID',
       'ChEBI ID', 'PharmGKB ID', 'HET ID', 'UniProt ID', 'UniProt Title',
       'GenBank ID', 'DPD ID', 'RxList Link', 'Pdrhealth Link', 'Wikipedia ID',
       'Drugs.com Link', 'NDC ID', 'ChemSpider ID', 'BindingDB ID', 'TTD ID'],
      dtype='object')
(2594, 2)


,DrugBank ID,KEGG Drug ID
0,DB00006,D03136
1,DB00014,D00573
2,DB00027,D04369
3,DB00035,D00291
4,DB00050,D01685


In [19]:
print('#Items = {}, # DrugBank ID = {}, # KEGG ID = {}'.format(DB_drug_links.shape[0],
                                                               len(set(DB_drug_links.iloc[:,0])),
                                                               len(set(DB_drug_links.iloc[:,1])) - 1)) # - nan

assert DB_drug_links.shape[0] == len(set(DB_drug_links.iloc[:,0])) # 说明DB ID都是不相同的
assert DB_drug_links.dropna(axis = 0, how = 'any').shape[0] ==  len(set(DB_drug_links.iloc[:,1])) - 1 # 说明有的DB没有KEGG Drug ID

#Items = 2594, # DrugBank ID = 2594, # KEGG ID = 1584


######  DB_DTIs_uniprot_links

In [20]:
DB_DTIs_uniprot_links = pd.read_csv('./0_database_file/uniprot links.csv')
print(DB_DTIs_uniprot_links.shape)

DB_DTIs_uniprot_links = DB_DTIs_uniprot_links[DB_DTIs_uniprot_links['Type'] == 'SmallMoleculeDrug'][['DrugBank ID', 'UniProt ID']].reset_index(drop = True)
print(DB_DTIs_uniprot_links.shape)
DB_DTIs_uniprot_links.head()

(10364, 5)
(9613, 2)


,DrugBank ID,UniProt ID
0,DB00006,P00734
1,DB00014,P22888
2,DB00014,P30968
3,DB00035,P30518
4,DB00035,P37288


In [21]:
print('#DTIs = {}, # Drug DrugBank ID = {}, # Target Uniprot ID = {}'.format(DB_DTIs_uniprot_links.shape[0],
                                                                         len(set(DB_DTIs_uniprot_links.iloc[:,0])),
                                                                         len(set(DB_DTIs_uniprot_links.iloc[:,1])))) # - nan

assert DB_DTIs_uniprot_links.shape[0] == DB_DTIs_uniprot_links.dropna(axis = 0, how = 'any').shape[0] # 说明每一个条目是一个DTIs，没有重叠或缺失

#DTIs = 9613, # Drug DrugBank ID = 1812, # Target Uniprot ID = 2542


### 处理：合并/匹配

###### 合并DrugBank ID， KEGG Drug ID，Target Uniprot ID  

In [22]:
DB_links = pd.merge(DB_drug_links, DB_DTIs_uniprot_links, left_on = 'DrugBank ID', right_on = 'DrugBank ID', how = 'right')

print('#DTIs = {}, #Drug-DrugBank ID = {}, #Drug-KEGG Drug ID = {}, #Target Uniprot ID = {}'.format(DB_links.shape[0],
                                                                                            len(set(DB_links.iloc[:,0])),
                                                                                            len(set(DB_links.iloc[:,1])),
                                                                                            len(set(DB_links.iloc[:,2]))))
print('去掉所有存在NaN的行, #DTIs = {}'.format(DB_links.dropna(axis = 0, how = 'any').shape[0]))

assert DB_links.shape == DB_links[~DB_links['UniProt ID'].isnull()].shape # 说明DTIs都包含在内，没有丢失DTIs
assert len(set(DB_DTIs_uniprot_links.iloc[:,0])) == len(set(DB_links.iloc[:,0])) # 没有丢失DTIs
assert len(set(DB_DTIs_uniprot_links.iloc[:,1])) == len(set(DB_links.iloc[:,2])) # 没有丢失DTIs

DB_links.head() # KEGG Drug ID存在缺失值

#DTIs = 9613, #Drug-DrugBank ID = 1812, #Drug-KEGG Drug ID = 1325, #Target Uniprot ID = 2542
去掉所有存在NaN的行, #DTIs = 6663


,DrugBank ID,KEGG Drug ID,UniProt ID
0,DB00006,D03136,P00734
1,DB00014,D00573,P22888
2,DB00014,D00573,P30968
3,DB00035,D00291,P30518
4,DB00035,D00291,P37288


##### 匹配NR_hsa_ID

In [23]:
DB_DTIs_3_ID = pd.merge(DB_links, hsa_uniprot_id_df, left_on = 'UniProt ID', right_on = 'UniProt ID', how = 'inner')[['DrugBank ID', 'KEGG Drug ID', 'KEGG_hsa_ID']]
print(DB_DTIs_3_ID.shape)

DB_DTIs_3_ID = DB_DTIs_3_ID.drop_duplicates(['DrugBank ID', 'KEGG Drug ID', 'KEGG_hsa_ID'])
print(DB_DTIs_3_ID.shape)

DB_DTIs_3_ID.head()

(461, 3)
(461, 3)


,DrugBank ID,KEGG Drug ID,KEGG_hsa_ID
0,DB00132,NaN,5465
1,DB00159,D08061,5465
2,DB00313,D00399,5465
3,DB00328,D00141,5465
4,DB00412,D00596,5465


In [24]:
print('#DTIs = {}, #Drugs-DrugBank ID = {}, #Drugs-KEGG Drug ID = {}, # Targets-KEGG hsa ID = {}'.format(DB_DTIs_3_ID.shape[0],
                                                                                 len(set(DB_DTIs_3_ID.iloc[:,0])),
                                                                                 len(set(DB_DTIs_3_ID.iloc[:,1])) - 1, # - nan
                                                                                 len(set(DB_DTIs_3_ID.iloc[:,2])),))

#DTIs = 461, #Drugs-DrugBank ID = 246, #Drugs-KEGG Drug ID = 190, # Targets-KEGG hsa ID = 31


##### 与之前的 KEGG_Yamanishi_merged_DTIs进行合并

In [25]:
print('KEGG_Yamanishi_merged_NR_DTIs: #DTIs = {}, #Drugs = {}, #Targets = {}'.format(KEGG_Yamanishi_merged_DTIs.shape[0],
                                                                             len(set(KEGG_Yamanishi_merged_DTIs.iloc[:,1])),
                                                                             len(set(KEGG_Yamanishi_merged_DTIs.iloc[:,0]))))
KEGG_Yamanishi_merged_DTIs.iloc[1,:]

KEGG_Yamanishi_merged_NR_DTIs: #DTIs = 632, #Drugs = 470, #Targets = 29


KEGG_hsa_ID       7067
KEGG Drug ID    D01011
Name: 1, dtype: object

In [26]:
DB_DTIs_2_ID_drugbankDBid = DB_DTIs_3_ID[DB_DTIs_3_ID['KEGG Drug ID'].isnull()][['DrugBank ID', 'KEGG_hsa_ID']].reset_index(drop = True)

DB_DTIs_2_ID_keggDid = DB_DTIs_3_ID[~DB_DTIs_3_ID['KEGG Drug ID'].isnull()][['KEGG Drug ID', 'KEGG_hsa_ID']].reset_index(drop = True)

assert DB_DTIs_2_ID_drugbankDBid.shape[0] + DB_DTIs_2_ID_keggDid.shape[0] == DB_DTIs_3_ID.shape[0]

print('DTIs: #DrugBank DB ID & KEGG hsa ID = {}, #KEGG Drug ID & KEGG hsa ID = {}'.format(DB_DTIs_2_ID_drugbankDBid.shape[0], 
                                                                                          DB_DTIs_2_ID_keggDid.shape[0]))

DTIs: #DrugBank DB ID & KEGG hsa ID = 107, #KEGG Drug ID & KEGG hsa ID = 354


In [27]:
KEGG_Yamanishi_DB_merged_DTIs_keggDid = pd.concat([DB_DTIs_2_ID_keggDid, KEGG_Yamanishi_merged_DTIs], sort = False).drop_duplicates().reset_index(drop = True)

print('在KEGG Drug ID & KEGG hsa ID的DTI中：#DTIs = {}, #Drugs = {}, #Targets = {}'.format(
                                                    KEGG_Yamanishi_DB_merged_DTIs_keggDid.shape[0],
                                                    len(set(KEGG_Yamanishi_DB_merged_DTIs_keggDid.iloc[:,0])),
                                                    len(set(KEGG_Yamanishi_DB_merged_DTIs_keggDid.iloc[:,1]))))

在KEGG Drug ID & KEGG hsa ID的DTI中：#DTIs = 814, #Drugs = 536, #Targets = 30


In [28]:
print('在DrugBank DB ID & KEGG hsa ID的DTI中：#DTIs = {}, #Drugs = {}, #Targets = {}'.format(
                                                    DB_DTIs_2_ID_drugbankDBid.shape[0],
                                                    len(set(DB_DTIs_2_ID_drugbankDBid.iloc[:,0])),
                                                    len(set(DB_DTIs_2_ID_drugbankDBid.iloc[:,1]))))

在DrugBank DB ID & KEGG hsa ID的DTI中：#DTIs = 107, #Drugs = 56, #Targets = 24


In [29]:
print('在KEGG和DrugBank中，全部的DTI：#DTIs = {}, # Drugs = {}, #Targets = {}'.format(
    KEGG_Yamanishi_DB_merged_DTIs_keggDid.shape[0] + DB_DTIs_2_ID_drugbankDBid.shape[0],
    len(set(DB_DTIs_2_ID_drugbankDBid.iloc[:,0]).union(set(KEGG_Yamanishi_DB_merged_DTIs_keggDid.iloc[:,0]))),
    len(set(DB_DTIs_2_ID_drugbankDBid.iloc[:,1]).union(set(KEGG_Yamanishi_DB_merged_DTIs_keggDid.iloc[:,1])))))

在KEGG和DrugBank中，全部的DTI：#DTIs = 921, # Drugs = 592, #Targets = 33


In [30]:
KEGG_id_drugs = set(KEGG_Yamanishi_DB_merged_DTIs_keggDid.iloc[:,0])
DB_id_drugs = set(DB_DTIs_2_ID_drugbankDBid.iloc[:,0])

# 检查其余的DTIs（由Drug爬虫得到的）是否真实存在

In [31]:
file1 = np.load('./2_output_file/1.4_NR_KEGG_UNIPROT_DRUGBANK_DTI.npz')
KEGG_DTIs_version1 = file1['drug_in_kegg_DTIs__D']
del file1
KEGG_DTIs_version1.shape, len(set(KEGG_DTIs_version1[:, 0])), len(set(KEGG_DTIs_version1[:, 1]))

((871, 2), 30, 536)

In [32]:
assert set(KEGG_Yamanishi_DB_merged_DTIs_keggDid.iloc[:,1]) - set(KEGG_DTIs_version1[:, 0]) == set()
assert set(KEGG_Yamanishi_DB_merged_DTIs_keggDid.iloc[:,0]) - set(KEGG_DTIs_version1[:, 1]) == set()
assert set(KEGG_DTIs_version1[:, 0]) - set(KEGG_Yamanishi_DB_merged_DTIs_keggDid.iloc[:,1]) == set()
assert set(KEGG_DTIs_version1[:, 1]) - set(KEGG_Yamanishi_DB_merged_DTIs_keggDid.iloc[:,0]) == set()

In [33]:
check_df = pd.concat([KEGG_Yamanishi_DB_merged_DTIs_keggDid[['KEGG_hsa_ID', 'KEGG Drug ID']],
                      pd.DataFrame(KEGG_DTIs_version1, columns = ['KEGG_hsa_ID', 'KEGG Drug ID'])]).drop_duplicates(['KEGG_hsa_ID', 'KEGG Drug ID'],
                                                                                                         keep = False).reset_index(drop = True)
assert check_df.shape[0] + KEGG_Yamanishi_DB_merged_DTIs_keggDid.shape[0] == KEGG_DTIs_version1.shape[0]
check_df.head()

,KEGG_hsa_ID,KEGG Drug ID
0,367,D00312
1,2104,D00966
2,8856,D00966
3,367,D00966
4,190,D02592


In [138]:
check_df = pd.concat([final_merged_DTIs_keggDid[['KEGG_hsa_ID', 'KEGG Drug ID']],
                      pd.DataFrame(KEGG_DTIs_version1, columns = ['KEGG_hsa_ID', 'KEGG Drug ID'])]).drop_duplicates(['KEGG_hsa_ID', 'KEGG Drug ID'],
                                                                                                         keep = False).reset_index(drop = True)
check_df #在网站里得到验证

,KEGG_hsa_ID,KEGG Drug ID
0,2100,D05987


# 对Drug进行爬虫

In [34]:
def scrapy_kegg(drug_id):
    
    url = 'https://www.kegg.jp/dbget-bin/www_bget?dr:' + drug_id
    html = requests.get(url).content.decode()
    soup = BeautifulSoup(html,'lxml')

    for idx, item in enumerate(soup.select('table > tr')):
        if item.text[:6] == 'Target':
            break
    target = soup.select('table > tr')[idx].text
    target_id = [item.replace(' ',',hsa').replace('HSA:','hsa').split(',') for item in re.findall(re.compile('HSA:[0-9\s]+'),target)]
    target_id_list = []
    for l in target_id:
        target_id_list.extend(l)
    target_id_list = list(set(target_id_list))

    database_href = soup.select('td > table > tr > td > a')
    chembl_link, drugbank_link = [], []
    for jtem in database_href:
        try:
            ljnk = jtem.get('href')
            if ljnk[:54] == 'https://www.ebi.ac.uk/chembldb/compound/inspect/CHEMBL':
                chembl_link.append(jtem.get('href'))
            elif ljnk[:29] == 'https://www.drugbank.ca/drugs':
                drugbank_link.append(jtem.get('href'))
        except:
            continue        
        
    return target_id_list, chembl_link, drugbank_link


In [35]:
def scrapy_uniprot(uniprot_link_):
    html_uniprot = requests.get(uniprot_link_).content.decode()
    soup_uniprot = BeautifulSoup(html_uniprot, 'lxml')
    content = soup_uniprot.select('#sequences > table.databaseTable.GENOME > tr > td > a')
    kegg_id = []
    for item in content:
        if item.get('href')[:45] == 'https://www.genome.jp/dbget-bin/www_bget?hsa:':
            kegg_id.append(item.text.replace(':',''))
    
    return kegg_id

In [36]:
def scrapy_drugbank(drugbank_link_):
    html = requests.get(drugbank_link_).content.decode()
    soup = BeautifulSoup(html,'lxml')

    uniprot_link = soup.select('div[class="bond-list-container targets"]')[0]
    uniprot_link = re.findall(re.compile('http://www.uniprot.org/uniprot/.[A-Z0-9]*'), str(uniprot_link))
    return uniprot_link

In [37]:
def summary_scrapy_hsa_from_drugbank(drug_id, kegg_details_dict):
    
    drugbank_link = kegg_details_dict[drug_id][2]
    if drugbank_link:
        for drugbank_link_ in drugbank_link:
            uniprot_link = scrapy_drugbank(drugbank_link_)
            kegg_id = [scrapy_uniprot(link) for link in uniprot_link]
            
            target_id_list = []
            for l in kegg_id:
                target_id_list.extend(l)
            target_id_list = list(set(target_id_list))
            
        return uniprot_link, target_id_list
    else:
        return [], []

In [51]:
def summary_scrapy_hsa_from_drugbank_DBid(DB_drug_id):
    
    drugbank_link = 'https://www.drugbank.ca/drugs/' + DB_drug_id
    
    uniprot_link = scrapy_drugbank(drugbank_link)
    kegg_id = [scrapy_uniprot(link) for link in uniprot_link]

    target_id_list = []
    for l in kegg_id:
        target_id_list.extend(l)
    target_id_list = list(set(target_id_list))

    return uniprot_link, target_id_list


### 爬虫 

###### KEGG 

In [38]:
drug_id_list, nr_details = [], []
error_drug_list = []

for drug_id in tqdm(list(set(KEGG_id_drugs))):
    
    try:

        nr_details.append(scrapy_kegg(drug_id))
        drug_id_list.append(drug_id)

    except:
        error_drug_list.append(drug_id)
        print('ERROR: ', drug_id)
        

  5%|▍         | 25/536 [07:50<13:40:07, 96.30s/it]

ERROR:  D07221


 13%|█▎        | 72/536 [22:52<13:04:40, 101.47s/it]

ERROR:  D03691


 18%|█▊        | 96/536 [39:32<25:17:42, 206.96s/it]

...

 53%|█████▎    | 282/536 [1:29:36<7:05:34, 100.53s/it] 

ERROR:  D02554


 69%|██████▉   | 370/536 [1:50:39<1:04:08, 23.18s/it] 

ERROR:  D07536


100%|██████████| 536/536 [2:18:19<00:00, 15.48s/it]  


In [40]:
error_drug_list2 = []
for drug_id in tqdm(error_drug_list):
    
    try:

        nr_details.append(scrapy_kegg(drug_id))
        drug_id_list.append(drug_id)

    except:
        error_drug_list2.append(drug_id)
        print('ERROR: ', drug_id)

100%|██████████| 6/6 [00:40<00:00,  6.77s/it]


In [42]:
nr_kegg_details_dict = dict(zip(drug_id_list,nr_details))
nr_kegg_details_dict

{'D05116': (['hsa367'],
  ['https://www.ebi.ac.uk/chembldb/compound/inspect/CHEMBL2104906'],
  ['https://www.drugbank.ca/drugs/DB00984']),
 'D05020': (['hsa4306'],
  ['https://www.ebi.ac.uk/chembldb/compound/inspect/CHEMBL3989628'],
  []),
 'D03551': (['hsa367', 'hsa2099', 'hsa2100'],
  ['https://www.ebi.ac.uk/chembldb/compound/inspect/CHEMBL2105974'],
  []),
 'D00751': (['hsa2908'],
  ['https://www.ebi.ac.uk/chembldb/compound/inspect/CHEMBL1201081',
   'https://www.ebi.ac.uk/chembldb/compound/inspect/CHEMBL346476'],
  ['https://www.drugbank.ca/drugs/DB00959']),
 'D00859': ([],
  ['https://www.ebi.ac.uk/chembldb/compound/inspect/CHEMBL496'],
  ['https://www.drugbank.ca/drugs/DB00756']),
 'D06201': (['hsa2908'],
  ['https://www.ebi.ac.uk/chembldb/compound/inspect/CHEMBL2105543'],
  []),
 'D05739': (['hsa5468'],
  ['https://www.ebi.ac.uk/chembldb/compound/inspect/CHEMBL2104753'],
  []),
 'D04467': (['hsa2908'],
  ['https://www.ebi.ac.uk/chembldb/compound/inspect/CHEMBL3989663'],
  []),
 

##### DrugBank 

In [43]:
db_drug_id_list, db_nr_details = [], []
db_error_drug_list = []

for drug_id in tqdm(KEGG_id_drugs):
    
    try:

        db_nr_details.append(summary_scrapy_hsa_from_drugbank(drug_id, nr_kegg_details_dict))
        db_drug_id_list.append(drug_id)

    except:
        
        db_error_drug_list.append(drug_id)
        print('ERROR: ', drug_id)


 17%|█▋        | 89/536 [24:57<4:25:26, 35.63s/it]

ERROR:  D00395


 29%|██▉       | 158/536 [41:43<3:51:11, 36.70s/it]

ERROR:  D00882


 48%|████▊     | 255/536 [58:49<1:18:46, 16.82s/it]

...

 91%|█████████ | 489/536 [2:02:33<56:35, 72.25s/it]  

ERROR:  D03658


 98%|█████████▊| 523/536 [2:14:47<06:20, 29.26s/it]

ERROR:  D01689


100%|██████████| 536/536 [2:16:39<00:00, 15.30s/it]


In [44]:
db_error_drug_list2 = []

for drug_id in tqdm(db_error_drug_list):
    
    try:

        db_nr_details.append(summary_scrapy_hsa_from_drugbank(drug_id, nr_kegg_details_dict))
        db_drug_id_list.append(drug_id)

    except:
        
        db_error_drug_list2.append(drug_id)
        print('ERROR: ', drug_id)
        
nr_db_details_dict = dict(zip(db_drug_id_list,db_nr_details))
nr_db_details_dict

 60%|██████    | 3/5 [02:56<02:01, 60.55s/it]

ERROR:  D03144


100%|██████████| 5/5 [07:21<00:00, 88.22s/it]

ERROR:  D01689


{'D05116': (['http://www.uniprot.org/uniprot/P10275'], ['hsa367']),
 'D05020': ([], []),
 'D03551': ([], []),
 'D00751': (['http://www.uniprot.org/uniprot/P04150'], ['hsa2908']),
 'D00859': (['http://www.uniprot.org/uniprot/P06149',
   'http://www.uniprot.org/uniprot/O14521',
   'http://www.uniprot.org/uniprot/P00367',
   'http://www.uniprot.org/uniprot/P03372'],
  ['hsa2746', 'hsa6392', 'hsa2099']),
 'D06201': ([], []),
 'D05739': ([], []),
 'D04467': ([], []),
 'D00596': (['http://www.uniprot.org/uniprot/P37231',
   'http://www.uniprot.org/uniprot/O60488',
   'http://www.uniprot.org/uniprot/Q07869',
   'http://www.uniprot.org/uniprot/Q03181',
   'http://www.uniprot.org/uniprot/P19793',
   'http://www.uniprot.org/uniprot/P28702',
   'http://www.uniprot.org/uniprot/P48443'],
  ['hsa6257',
   'hsa5468',
   'hsa6258',
   'hsa6256',
   'hsa5467',
   'hsa5465',
   'hsa2182']),
 'D01853': ([], []),
 'D01239': (['http://www.uniprot.org/uniprot/P04150'], ['hsa2908']),
 'D11336': ([], []),
 'D

###### DrugBank DB drug ID在DrugBank的爬虫 

In [61]:
db_error_drug_list3 = []

for drug_id in tqdm(list(DB_id_drugs)[1:]): # [1:]之前不小心break了，为了防止重复
    
    try:
        
        db_nr_details.append(summary_scrapy_hsa_from_drugbank_DBid(drug_id))
        db_drug_id_list.append(drug_id)
        

    except:
        
        db_error_drug_list3.append(drug_id)
        print('ERROR: ', drug_id)
        
# nr_db_details_dict = dict(zip(db_drug_id_list,db_nr_details))
# nr_db_details_dict

 55%|█████▍    | 30/55 [58:08<54:58, 131.92s/it]  

ERROR:  DB13946


 71%|███████   | 39/55 [1:05:01<18:31, 69.46s/it]

ERROR:  DB00132


100%|██████████| 55/55 [1:28:19<00:00, 96.36s/it] 

ERROR:  DB13155


In [63]:
db_error_drug_list4 = []

for drug_id in tqdm(db_error_drug_list3): # [1:]之前不小心break了，为了防止重复
    
    try:
        
        db_nr_details.append(summary_scrapy_hsa_from_drugbank_DBid(drug_id))
        db_drug_id_list.append(drug_id)
        

    except:
        
        db_error_drug_list4.append(drug_id)
        print('ERROR: ', drug_id)
        


100%|██████████| 3/3 [01:51<00:00, 37.07s/it]


In [64]:
nr_db_details_dict = dict(zip(db_drug_id_list,db_nr_details))
nr_db_details_dict

{'D05116': (['http://www.uniprot.org/uniprot/P10275'], ['hsa367']),
 'D05020': ([], []),
 'D03551': ([], []),
 'D00751': (['http://www.uniprot.org/uniprot/P04150'], ['hsa2908']),
 'D00859': (['http://www.uniprot.org/uniprot/P06149',
   'http://www.uniprot.org/uniprot/O14521',
   'http://www.uniprot.org/uniprot/P00367',
   'http://www.uniprot.org/uniprot/P03372'],
  ['hsa2746', 'hsa6392', 'hsa2099']),
 'D06201': ([], []),
 'D05739': ([], []),
 'D04467': ([], []),
 'D00596': (['http://www.uniprot.org/uniprot/P37231',
   'http://www.uniprot.org/uniprot/O60488',
   'http://www.uniprot.org/uniprot/Q07869',
   'http://www.uniprot.org/uniprot/Q03181',
   'http://www.uniprot.org/uniprot/P19793',
   'http://www.uniprot.org/uniprot/P28702',
   'http://www.uniprot.org/uniprot/P48443'],
  ['hsa6257',
   'hsa5468',
   'hsa6258',
   'hsa6256',
   'hsa5467',
   'hsa5465',
   'hsa2182']),
 'D01853': ([], []),
 'D01239': (['http://www.uniprot.org/uniprot/P04150'], ['hsa2908']),
 'D11336': ([], []),
 'D

### 处理结果 

In [73]:
len(nr_db_details_dict), len(nr_kegg_details_dict)

(590, 536)

In [69]:
scrapy_kegg_DTIs = []
for drug, value in nr_kegg_details_dict.items():    
    scrapy_kegg_DTIs.extend(list(product([drug], nr_kegg_details_dict[drug][0])))

scrapy_kegg_DTIs = np.array(scrapy_kegg_DTIs)

print('For example (i.e. scrapy_kegg_DTIs): the first = {}, the end = {}'.format(scrapy_kegg_DTIs[0], 
                                                                                 scrapy_kegg_DTIs[-1]))
print('In KEGG from all-drugs with their all targets: #DTIs = {}, #Drugs = {}, #Targets = {}'.format(len(scrapy_kegg_DTIs),
                                                                                                         len(set(scrapy_kegg_DTIs[:, 0])),
                                                                                                         len(set(scrapy_kegg_DTIs[:, 1]))))

#--------------------------------------------------------------

NR_scrapy_kegg_DTIs = []
for drug, target in tqdm(scrapy_kegg_DTIs):
    
    if target[3:] in list(Union_Targets):
        NR_scrapy_kegg_DTIs.append([drug, target[3:]])

NR_scrapy_kegg_DTIs = np.array(NR_scrapy_kegg_DTIs)

print('In KEGG from NR-class: # DTIs = {}, #Drugs = {}, #Targets = {}'.format(len(NR_scrapy_kegg_DTIs),
                                                                              len(set(NR_scrapy_kegg_DTIs[:, 0])),
                                                                              len(set(NR_scrapy_kegg_DTIs[:, 1]))))
print('For example (i.e. NR_scrapy_kegg_DTIs): the first = {}, the end = {}'.format(NR_scrapy_kegg_DTIs[0],
                                                                                    NR_scrapy_kegg_DTIs[-1]))

100%|██████████| 827/827 [00:00<00:00, 116940.51it/s]

For example (i.e. scrapy_kegg_DTIs): the first = ['D05116' 'hsa367'], the end = ['D07536' 'hsa367']
In KEGG from all-drugs with their all targets: #DTIs = 827, #Drugs = 501, #Targets = 112
In KEGG from NR-class: # DTIs = 603, #Drugs = 461, #Targets = 19
For example (i.e. NR_scrapy_kegg_DTIs): the first = ['D05116' '367'], the end = ['D07536' '367']


In [70]:
scrapy_drugbank_DTIs = []
for drug, value in nr_db_details_dict.items():    
    scrapy_drugbank_DTIs.extend(list(product([drug], nr_db_details_dict[drug][-1])))

scrapy_drugbank_DTIs = np.array(scrapy_drugbank_DTIs)
print('For example (i.e. scrapy_drugbank_DTIs): the first = {}, the end = {}'.format(scrapy_drugbank_DTIs[0], 
                                                                                     scrapy_drugbank_DTIs[-1]))
print('In DrugBank from all-drugs with their all targets: #DTIs = {}, #Drugs = {}, #Targets = {}'.format(len(scrapy_drugbank_DTIs),
                                                                                                         len(set(scrapy_drugbank_DTIs[:, 0])),
                                                                                                         len(set(scrapy_drugbank_DTIs[:, 1]))))

#--------------------------------------------------------------

NR_scrapy_drugbank_DTIs = []
for drug, target in tqdm(scrapy_drugbank_DTIs):
    
    if target[3:] in list(Union_Targets):
        NR_scrapy_drugbank_DTIs.append([drug, target[3:]])

NR_scrapy_drugbank_DTIs = np.array(NR_scrapy_drugbank_DTIs)

print('In DrugBank from NR-class: # DTIs = {}, #Drugs = {}, #Targets = {}'.format(len(NR_scrapy_drugbank_DTIs),
                                                                                  len(set(NR_scrapy_drugbank_DTIs[:, 0])),
                                                                                  len(set(NR_scrapy_drugbank_DTIs[:, 1]))))
print('For example (i.e. NR_scrapy_drugbank_DTIs): the first = {}, the end = {}'.format(NR_scrapy_drugbank_DTIs[0],
                                                                                        NR_scrapy_drugbank_DTIs[-1]))

100%|██████████| 1715/1715 [00:00<00:00, 137480.05it/s]

For example (i.e. scrapy_drugbank_DTIs): the first = ['D05116' 'hsa367'], the end = ['DB13155' 'hsa367']
In DrugBank from all-drugs with their all targets: #DTIs = 1715, #Drugs = 294, #Targets = 525
In DrugBank from NR-class: # DTIs = 550, #Drugs = 290, #Targets = 31
For example (i.e. NR_scrapy_drugbank_DTIs): the first = ['D05116' '367'], the end = ['DB13155' '367']


### 合并 

In [111]:
scrapy_DTIs_df = pd.concat([pd.DataFrame(NR_scrapy_drugbank_DTIs, columns = ['Drug', 'Target']), 
                            pd.DataFrame(NR_scrapy_kegg_DTIs, columns = ['Drug', 'Target'])]).reset_index(drop = True).\
                 drop_duplicates(['Drug', 'Target']).reset_index(drop = True)

In [125]:
scrapy_DTIs_keggDid = []
scrapy_DTIs_DrugBankDBid = []
for item, jtem in np.array(scrapy_DTIs_df):

    if item[:2] == 'DB':
        scrapy_DTIs_DrugBankDBid.append([item, jtem])
    else:
        scrapy_DTIs_keggDid.append([item, jtem])
        
scrapy_DTIs_DrugBankDBid = np.array(scrapy_DTIs_DrugBankDBid)
scrapy_DTIs_keggDid = np.array(scrapy_DTIs_keggDid)

print('In DrugBank DB id: #DTIs = {}, #Drugs = {}, #Targets = {}'.format(scrapy_DTIs_DrugBankDBid.shape[0],
                                                                         len(set(scrapy_DTIs_DrugBankDBid[:, 0])),
                                                                         len(set(scrapy_DTIs_DrugBankDBid[:, 1]))))
print('In KEGG BRITE D id: #DTIs = {}, #Drugs = {}, #Targets = {}'.format(scrapy_DTIs_keggDid.shape[0],
                                                                          len(set(scrapy_DTIs_keggDid[:, 0])),
                                                                          len(set(scrapy_DTIs_keggDid[:, 1]))))
print('#ALL DTIs = {}， #ALL Drugs = {}, #ALL Targets = {}'.format(scrapy_DTIs_DrugBankDBid.shape[0] + scrapy_DTIs_keggDid.shape[0], 
                                                                   len(set(scrapy_DTIs_keggDid[:, 0]).union(set(scrapy_DTIs_DrugBankDBid[:, 0]))),
                                                                   len(set(scrapy_DTIs_keggDid[:, 1]).union(set(scrapy_DTIs_DrugBankDBid[:, 1])))))

In DrugBank DB id: #DTIs = 107, #Drugs = 56, #Targets = 24
In KEGG BRITE D id: #DTIs = 828, #Drugs = 527, #Targets = 28
#ALL DTIs = 935， #ALL Drugs = 583, #ALL Targets = 31


In [126]:
scrapy_DTIs_DrugBankDBid = pd.DataFrame(scrapy_DTIs_DrugBankDBid, columns = ['DrugBank ID', 'KEGG_hsa_ID'])
scrapy_DTIs_keggDid = pd.DataFrame(scrapy_DTIs_keggDid, columns = ['KEGG Drug ID', 'KEGG_hsa_ID'])

In [132]:
final_merged_DTIs_keggDid = pd.concat([KEGG_Yamanishi_DB_merged_DTIs_keggDid, scrapy_DTIs_keggDid]).drop_duplicates(['KEGG Drug ID', 'KEGG_hsa_ID']).reset_index(drop = True)
final_merged_DTIs_DrugBankDBid = pd.concat([DB_DTIs_2_ID_drugbankDBid, scrapy_DTIs_DrugBankDBid]).drop_duplicates(['DrugBank ID', 'KEGG_hsa_ID']).reset_index(drop = True)

In [134]:
print('Final DTIs in KEGG BRITE D id: #DTIs = {}, #Drugs = {}, #Targets = {}'.format(final_merged_DTIs_keggDid.shape[0],
                                                                                     len(set(final_merged_DTIs_keggDid.iloc[:, 0])),
                                                                                     len(set(final_merged_DTIs_keggDid.iloc[:, 1]))))
print('Final DTIs in DrugBank DB id: #DTIs = {}, #Drugs = {}, #Targets = {}'.format(final_merged_DTIs_DrugBankDBid.shape[0],
                                                                                    len(set(final_merged_DTIs_DrugBankDBid.iloc[:, 0])),
                                                                                    len(set(final_merged_DTIs_DrugBankDBid.iloc[:, 1]))))
print('#ALL DTIs = {}， #ALL Drugs = {}, #ALL Targets = {}'.format(final_merged_DTIs_keggDid.shape[0] + final_merged_DTIs_DrugBankDBid.shape[0], 
                                                                   len(set(final_merged_DTIs_keggDid.iloc[:, 0]).union(set(final_merged_DTIs_DrugBankDBid.iloc[:, 0]))),
                                                                   len(set(final_merged_DTIs_keggDid.iloc[:, 1]).union(set(final_merged_DTIs_DrugBankDBid.iloc[:, 1])))))

Final DTIs in KEGG BRITE D id: #DTIs = 872, #Drugs = 536, #Targets = 30
Final DTIs in DrugBank DB id: #DTIs = 107, #Drugs = 56, #Targets = 24
#ALL DTIs = 979， #ALL Drugs = 592, #ALL Targets = 33


# 文件保存

In [149]:
np.savez_compressed('./2_output_file/2_NR_KEGGBRITE-Uniprot-DrugBank_DTIs.npz',
                    DTIs_keggDid = np.array(final_merged_DTIs_keggDid),
                    DTIs_DrugBankDBid = np.array(final_merged_DTIs_DrugBankDBid),
                    All_NR_targets_in_KEGG_BRITE = np.array(list(Union_Targets)),
                    Targets_in_all_DTIs = np.array(list(set(final_merged_DTIs_keggDid.iloc[:, 1]).union(set(final_merged_DTIs_DrugBankDBid.iloc[:, 1])))),
                    Drugs_in_all_DTIs = np.array(list(set(final_merged_DTIs_keggDid.iloc[:, 0]).union(set(final_merged_DTIs_DrugBankDBid.iloc[:, 0])))))